In [6]:
# # Convert Channels JSON to CSV

# import pandas as pd
# import json

# # Read JSON file
# with open('../data/channels.json') as f:
#     data = json.load(f)

# df = pd.json_normalize(data)

# # Rename the columns for clarity
# # df.columns = ['id', 'name', 'created', 'creator', 'is_archived', 'is_general', 'members', 'topic_value', 'topic_creator', 'purpose_value', 'purpose_creator']


# # Convert JSON to Pandas DataFrame
# df = pd.DataFrame(data)

# # # Convert Pandas DataFrame to CSV
# df.to_csv('../csv/channels.csv', index=False)

In [8]:
# import pandas as pd
# import json
# import os

# # Specify the directory containing JSON files
# json_directory = '../data/'

# # Specify the directory to save CSV files
# csv_directory = '../csv/'

# # Get a list of JSON files in the directory
# json_files = [f for f in os.listdir(json_directory) if f.endswith('.json')]

# # Process each JSON file and convert it to CSV
# for json_file in json_files:
#     json_file_path = os.path.join(json_directory, json_file)
    
#     # Read JSON file
#     with open(json_file_path) as f:
#         data = json.load(f)

#     # Convert JSON to Pandas DataFrame
#     df = pd.json_normalize(data)

#     # Convert Pandas DataFrame to CSV
#     csv_file = os.path.splitext(json_file)[0] + '.csv'  # Use the same file name, but with a CSV extension
#     csv_file_path = os.path.join(csv_directory, csv_file)
#     df.to_csv(csv_file_path, index=False)

#     print(f"Converted '{json_file}' to '{csv_file}'")


Converted 'canvases.json' to 'canvases.csv'
Converted 'channels.json' to 'channels.csv'
Converted 'file_conversations.json' to 'file_conversations.csv'
Converted 'integration_logs.json' to 'integration_logs.csv'
Converted 'users.json' to 'users.csv'


In [12]:
import pandas as pd
import json
import os
import shutil

# Specify the root directory containing JSON files and subdirectories
json_root_directory = '../data2/'

# Specify the root directory to save CSV files
csv_root_directory = '../csv2/'

# Function to convert JSON to CSV and preserve directory structure
def convert_json_to_csv(json_directory, csv_directory):
    for root, _, files in os.walk(json_directory):
        for json_file in files:
            if json_file.endswith('.json'):
                json_file_path = os.path.join(root, json_file)

                # Read JSON file
                with open(json_file_path, encoding='utf-8') as f:
                    data = json.load(f)

                # Convert JSON to Pandas DataFrame
                df = pd.json_normalize(data)

                # Create the CSV directory structure
                relative_path = os.path.relpath(root, json_directory)
                csv_subdirectory = os.path.join(csv_directory, relative_path)

                # Create the CSV subdirectory if it doesn't exist
                os.makedirs(csv_subdirectory, exist_ok=True)

                # Convert Pandas DataFrame to CSV
                csv_file = os.path.splitext(json_file)[0] + '.csv'
                csv_file_path = os.path.join(csv_subdirectory, csv_file)
                df.to_csv(csv_file_path, index=False)

                print(f"Converted '{json_file_path}' to '{csv_file_path}'")

# Call the function to convert JSON to CSV
convert_json_to_csv(json_root_directory, csv_root_directory)


Converted '../data2/canvases.json' to '../csv2/.\canvases.csv'
Converted '../data2/channels.json' to '../csv2/.\channels.csv'
Converted '../data2/dms.json' to '../csv2/.\dms.csv'
Converted '../data2/file_conversations.json' to '../csv2/.\file_conversations.csv'
Converted '../data2/groups.json' to '../csv2/.\groups.csv'
Converted '../data2/integration_logs.json' to '../csv2/.\integration_logs.csv'
Converted '../data2/mpims.json' to '../csv2/.\mpims.csv'
Converted '../data2/users.json' to '../csv2/.\users.csv'
Converted '../data2/D05RLN07QS1\2023-09-10.json' to '../csv2/D05RLN07QS1\2023-09-10.csv'
Converted '../data2/D05RLN07QS1\2023-09-15.json' to '../csv2/D05RLN07QS1\2023-09-15.csv'
Converted '../data2/D05RLN07QS1\2023-09-16.json' to '../csv2/D05RLN07QS1\2023-09-16.csv'
Converted '../data2/D05RQM36TK5\2023-09-10.json' to '../csv2/D05RQM36TK5\2023-09-10.csv'
Converted '../data2/D05RS3C1CQL\2023-09-20.json' to '../csv2/D05RS3C1CQL\2023-09-20.csv'
Converted '../data2/D05RS3C1CQL\2023-09-2

In [5]:
import zipfile
import json
import os
import pandas as pd

# Specify the path of the zip file
zip_file_path = '../Kentron Slack export Sep 10 2023 - Oct 15 2023.zip'

# Specify the directory to extract the zip file to
extract_directory = '../data3/raw/'

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_directory)

# Get a list of JSON files in the directory
json_files = [os.path.join(root, f) for root, _, files in os.walk(extract_directory) for f in files if f.endswith('.json')]

# Process each JSON file and convert it to a Pandas DataFrame
df_list = []
for json_file in json_files:
    with open(json_file, encoding='utf-8') as f:
        data = json.load(f)
    df = pd.json_normalize(data)
    df_list.append(df)

# Concatenate all DataFrames into a single DataFrame
df = pd.concat(df_list, ignore_index=True)

# Save the DataFrame to a CSV file
csv_file_path = '../data3/csv/normal1.csv'
df.to_csv(csv_file_path, index=False)

# Convert the DataFrame to a JSON string with proper structure
json_string = df.to_json(orient='records', indent=4)

# Save the JSON string to a file
json_file_path = '../data3/json/normal1.json'
with open(json_file_path, 'w') as f:
    f.write(json_string)


In [1]:
import zipfile
import json
import os
import pandas as pd
from flask import Flask, request
from flask_graphql import GraphQLView
import graphene

# Define the GraphQL schema
class Query(graphene.ObjectType):
    pass

class UploadZip(graphene.Mutation):
    class Arguments:
        file = graphene.Upload(required=True)

    ok = graphene.Boolean()
    csv_path = graphene.String()

    def mutate(self, info, file):
        # Specify the directory to extract the zip file to
        extract_directory = '../data4/raw/'

        # Extract the zip file
        with zipfile.ZipFile(file, 'r') as zip_ref:
            zip_ref.extractall(extract_directory)

        # Get a list of JSON files in the directory
        json_files = [os.path.join(root, f) for root, _, files in os.walk(extract_directory) for f in files if f.endswith('.json')]

        # Process each JSON file and convert it to a Pandas DataFrame
        df_list = []
        for json_file in json_files:
            with open(json_file, encoding='utf-8') as f:
                data = json.load(f)
            df = pd.json_normalize(data)
            df_list.append(df)

        # Concatenate all DataFrames into a single DataFrame
        df = pd.concat(df_list, ignore_index=True)

        # Save the DataFrame to a CSV file
        csv_file_path = '../data4/csv/normal1.csv'
        df.to_csv(csv_file_path, index=False)

        return UploadZip(ok=True, csv_path=csv_file_path)

class Mutation(graphene.ObjectType):
    upload_zip = UploadZip.Field()

schema = graphene.Schema(query=Query, mutation=Mutation)

# Create a Flask app
app = Flask(__name__)

# Add a route for the GraphQL endpoint
app.add_url_rule('/graphql', view_func=GraphQLView.as_view('graphql', schema=schema, graphiql=True))

if __name__ == '__main__':
    app.run()


ModuleNotFoundError: No module named 'flask_graphql'